Write a query to return Territory and corresponding Sales Growth. Compare growth between periods Q4-2021 vs Q3-2021.
If Territory (say T123) has Sales worth $100 in Q3-2021 and Sales worth $110 in Q4-2021, then the Sales Growth will be 10% [ i.e. = ((110 - 100)/100) * 100 ]
Output the ID of the Territory and the Sales Growth. Only output these territories that had any sales in both quarters.


In [0]:
df = spark.read.format("csv").option("header","True").load("dbfs:/FileStore/Question2/Question2.csv")
#df.show()

In [0]:
df_map=spark.read.format("csv").option("header","True").load("dbfs:/FileStore/Question2/map.csv")
#df_map.show()

In [0]:
from pyspark.sql.functions import quarter,year, concat_ws, when, col

df_quarter = df.withColumn("quarter",when(quarter("order_date") == 1, "Q1")\
                                .when(quarter("order_date") == 2, "Q2")\
                                .when(quarter("order_date") == 3, "Q3")\
                                .when(quarter("order_date") == 4, "Q4")
)
df_year_quarter = df_quarter.withColumn("quarter_year", concat_ws("-",col("quarter"),year("order_date")))
#df_year_quarter.show()

In [0]:
df_cust = df_year_quarter
df_join = df_cust.join(df_map, df_cust.cust_id == df_map.cust_id, "inner").select(df_map.territory_id,df_cust.order_value,df_cust.quarter_year)
#df_join.show()

In [0]:
from pyspark.sql.functions import sum
df_Q3_Q4 = df_join.filter((col("quarter_year") == 'Q3-2021') | (col("quarter_year") == 'Q4-2021'))
df_group = df_Q3_Q4.groupBy("territory_id").agg(sum(when(col("quarter_year") == "Q4-2021" , col("order_value"))).alias("Q4-2021"),sum(when(col("quarter_year") == "Q3-2021" , col("order_value"))).alias("Q3-2021")
                        )
df_group.show()

In [0]:
df_dropped = df_group.dropna("any", subset=["Q4-2021","Q3-2021"])
#df_dropped.show()

In [0]:
from pyspark.sql.functions import round
df_growth = df_dropped.withColumn("pert_growth",round(((col("Q4-2021")-col("Q3-2021"))/col("Q3-2021")) * 100, 2))
df_final = df_growth.select("territory_id","Q3-2021","Q4-2021","pert_growth")        
df_final.show()                       